In [1]:

# Update package lists quietly (requires sudo)
!sudo apt-get update -qq

!pip install gradio

# Install aria2 (which provides aria2c) and zip without interactive prompts and quietly (requires sudo)
# Using the correct package name 'aria2'
!sudo apt-get install -y -qq aria2 zip

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires

In [ ]:

# --- Imports ---
import os
import subprocess
import shlex
import time
import sys
import gradio as gr # Import Gradio
import uuid # To create unique download subdirs
import traceback # For detailed error logging in status
from google.colab import drive # Import Google Drive library

# ==============================================================================
# 1. Mount Google Drive (Run this *before* defining the app)
# ==============================================================================
DRIVE_MOUNT_POINT = '/content/drive'
DRIVE_EXPECTED_PATH = os.path.join(DRIVE_MOUNT_POINT, 'MyDrive') # Common path check
drive_mounted = False
print("--- Google Drive Mounting ---")
try:
    # Attempt to mount Google Drive
    drive.mount(DRIVE_MOUNT_POINT, force_remount=True) # force_remount can be helpful

    # Verify if the standard MyDrive path exists after mounting
    if os.path.exists(DRIVE_EXPECTED_PATH):
        drive_mounted = True
        print(f"✅ Google Drive successfully mounted at: {DRIVE_MOUNT_POINT}")
        print(f"✅ Found expected path: {DRIVE_EXPECTED_PATH}")
    else:
        # Mounted, but MyDrive isn't where expected? Or mount failed silently?
        print(f"⚠️ Google Drive mounted at {DRIVE_MOUNT_POINT}, but the standard '{DRIVE_EXPECTED_PATH}' path was not found.")
        print("   Please ensure your Drive structure is standard or adjust paths accordingly.")
        # We can still proceed, but warn the user. drive_mounted = True is okay here.
        drive_mounted = True

except ImportError:
    print("⚠️ 'google.colab.drive' not available. Running outside of Google Colab?")
    print("   Google Drive paths like '/content/drive/MyDrive/...' will NOT work.")
except Exception as e:
    print(f"❌ An error occurred during Google Drive mounting: {e}")
    print("   Google Drive paths like '/content/drive/MyDrive/...' will likely NOT work.")
print("-" * 30)

# ==============================================================================
# Helper Function: Find New Items (same as before)
# ==============================================================================
def find_new_items(directory, items_before):
    """Compares current directory contents with a previous set."""
    try:
        if not isinstance(items_before, set):
             items_before = set(items_before)
        # Add a small delay before listing, sometimes helps with GDriveFS consistency
        time.sleep(0.5)
        items_after = set(os.listdir(directory))
        new_items = items_after - items_before
        return list(new_items)
    except FileNotFoundError:
        return None
    except Exception as e:
        return None

# ==============================================================================
# Core Logic Function (Corrected Yielding - No changes needed here)
# ==============================================================================
def run_download_and_zip(
    torrent_file_obj, # Gradio File object
    download_directory_base,
    stop_seeding_after_download,
    auto_zip_after_download,
    output_zip_name_override
    ):

    status_log = ""
    final_zip_path = None # Path for the final zip file
    current_zip_output_update = gr.update(value=None, visible=False) # Initial state for file output

    # --- Helper to yield updates for both outputs ---
    def yield_update(log_message_add = "", new_zip_update=None):
        nonlocal status_log, current_zip_output_update
        status_log += log_message_add
        if new_zip_update is not None:
             current_zip_output_update = new_zip_update
        # Yield must contain value for each output component
        yield status_log, current_zip_output_update

    # --- Generator starts here ---
    try:
        # Add note about drive status at the beginning of the log
        if drive_mounted:
             yield from yield_update("ℹ️ Google Drive appears to be mounted.\n")
        else:
             yield from yield_update("⚠️ Google Drive does not appear to be mounted. Paths starting with '/content/drive/' may fail.\n")

        yield from yield_update("--- Setup and Validation ---\n")

        # Check for uploaded file
        if torrent_file_obj is None:
            yield from yield_update("❌ ERROR: No torrent file uploaded.\nPlease upload a .torrent file.\n")
            return status_log, gr.update(value=None, visible=False)

        torrent_file_path = torrent_file_obj.name
        yield from yield_update(f"ℹ️ Using uploaded torrent file: {os.path.basename(torrent_file_path)}\n")

        # Check if aria2c is installed
        if subprocess.run(["which", "aria2c"], capture_output=True, text=True).stdout.strip() == "":
            yield from yield_update("⏳ aria2c not found. Attempting to install...\n")
            install_process = subprocess.run(["apt-get", "update", "-qq"], capture_output=True, text=True)
            install_process = subprocess.run(["apt-get", "install", "-y", "-qq", "aria2c"], capture_output=True, text=True)
            if install_process.returncode == 0:
                yield from yield_update("✅ aria2c installed successfully.\n")
            else:
                err_msg = f"❌ Failed to install aria2c. Cannot proceed.\n   Error: {install_process.stderr}\n"
                yield from yield_update(err_msg)
                return status_log, gr.update(value=None, visible=False)
        else:
            yield from yield_update("✅ aria2c is already installed.\n")

        # Check if zip is installed
        zip_available = False
        if subprocess.run(["which", "zip"], capture_output=True, text=True).stdout.strip() != "":
             zip_available = True
             if auto_zip_after_download:
                  yield from yield_update("✅ zip utility is already installed.\n")
        elif auto_zip_after_download:
            yield from yield_update("⏳ zip utility not found. Attempting to install...\n")
            install_process = subprocess.run(["apt-get", "update", "-qq"], capture_output=True, text=True)
            install_process = subprocess.run(["apt-get", "install", "-y", "-qq", "zip"], capture_output=True, text=True)
            if install_process.returncode == 0:
                yield from yield_update("✅ zip installed successfully.\n")
                zip_available = True
            else:
                err_msg = f"❌ Failed to install zip utility. Cannot proceed with zipping.\n   Error: {install_process.stderr}\n⚠️ Zipping will be disabled for this run.\n"
                yield from yield_update(err_msg)
                auto_zip_after_download = False

        # --- Create a unique download directory for this run ---
        run_id = str(uuid.uuid4())[:8]
        if not download_directory_base:
            download_directory_base = "/content/downloads"
            yield from yield_update(f"ℹ️ No download directory specified, defaulting to temporary storage: {download_directory_base}\n")
        elif download_directory_base.startswith('/content/drive') and not drive_mounted:
             yield from yield_update(f"⚠️ WARNING: Specified download directory '{download_directory_base}' is on Google Drive, but Drive is not mounted. This will likely fail!\n")


        download_directory = os.path.join(download_directory_base, f"download_{run_id}")
        yield from yield_update(f"ℹ️ Download target directory for this run: {download_directory}\n")

        items_before_download = set()
        try:
            os.makedirs(download_directory, exist_ok=True)
            yield from yield_update(f"✅ Ensured download directory exists: {download_directory}\n")
            # Add a small delay specifically before listing on GDrive
            if download_directory.startswith('/content/drive'): time.sleep(1)
            items_before_download = set(os.listdir(download_directory))
            yield from yield_update(f"ℹ️ Found {len(items_before_download)} items in target directory before starting.\n")
        except OSError as e:
             # Check if it's a GDrive path error after mount warning
             if download_directory.startswith('/content/drive') and not drive_mounted:
                 err_msg = f"❌ ERROR: Failed to create/access GDrive path '{download_directory}' as expected (Drive not mounted).\n   Error details: {e}\n"
             else:
                err_msg = f"❌ ERROR: Could not create or access download directory: {download_directory}\n   Error details: {e}\n"
             yield from yield_update(err_msg)
             return status_log, gr.update(value=None, visible=False)
        except Exception as e:
            err_msg = f"❌ An unexpected error occurred during setup checking download directory: {e}\n"
            yield from yield_update(err_msg)
            return status_log, gr.update(value=None, visible=False)

        # Validate torrent file again
        if not os.path.isfile(torrent_file_path):
            yield from yield_update(f"❌ ERROR: Torrent file vanished after upload: {torrent_file_path}\n")
            return status_log, gr.update(value=None, visible=False)

        download_successful = False
        path_to_zip_item = None
        content_name = None

        # --- 2. Execute Download ---
        yield from yield_update("\n--- Starting Download ---\n" + "-" * 30 + "\n")

        command = [
            "aria2c", "--console-log-level=warn", "--summary-interval=5",
            "--human-readable=true", "-d", download_directory,
        ]
        if stop_seeding_after_download:
            command.append("--seed-time=0")
        command.append(torrent_file_path)

        process = None
        try:
            process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True)
            for line in process.stdout:
                yield from yield_update(line)

            process.wait()
            exit_code = process.returncode
            yield from yield_update("-" * 30 + "\n")

            if exit_code == 0:
                yield from yield_update(f"✅ Download command finished successfully (Exit Code 0).\nFiles should be in: {download_directory}\n")
                download_successful = True
                # Longer wait for GDrive sync might be needed
                wait_time = 10 if download_directory.startswith('/content/drive') else 5
                yield from yield_update(f"⏳ Waiting {wait_time} seconds for filesystem sync...\n")
                time.sleep(wait_time)
            else:
                yield from yield_update(f"❌ Download command finished with errors (Exit Code: {exit_code}). Check log above.\n")

        except FileNotFoundError:
            yield from yield_update(f"\n❌ ERROR: 'aria2c' command not found. Installation might have failed.\n" + "-" * 30 + "\n")
        except Exception as e:
            yield from yield_update(f"\n❌ An error occurred while running the download process: {e}\n" + "-" * 30 + "\n")
            yield from yield_update(f"Traceback:\n{traceback.format_exc()}\n")
        finally:
            if process and process.poll() is None:
                yield from yield_update("\n⏳ Terminating potentially running download process...\n")
                process.terminate()
                try: process.wait(timeout=5)
                except subprocess.TimeoutExpired: process.kill()
                yield from yield_update("✅ Process terminated.\n")


        # --- 3. Determine Downloaded Content ---
        if download_successful:
            yield from yield_update("\n--- Determining Downloaded Content ---\n")
            # Add extra wait before finding items on GDrive
            if download_directory.startswith('/content/drive'):
                 yield from yield_update("⏳ Extra wait for GDrive listing...\n"); time.sleep(5)
            new_items = find_new_items(download_directory, items_before_download)

            if new_items is None:
                yield from yield_update(f"❌ Could not determine new items (directory listing failed): {download_directory}\n")
                download_successful = False
            elif len(new_items) == 1:
                content_name = new_items[0]
                potential_path = os.path.join(download_directory, content_name)
                yield from yield_update(f"ℹ️ Identified potential new item: '{content_name}'\n")
                if os.path.exists(potential_path):
                    path_to_zip_item = potential_path
                    yield from yield_update(f"✅ Successfully confirmed existence of: '{path_to_zip_item}'\n")
                else:
                    yield from yield_update(f"⚠️ Warning: Identified item '{potential_path}' not found immediately. Waiting 15s (longer for GDrive)...\n")
                    time.sleep(15)
                    if os.path.exists(potential_path):
                        path_to_zip_item = potential_path
                        yield from yield_update(f"✅ Found item after waiting: '{path_to_zip_item}'\n")
                    else:
                        yield from yield_update(f"❌ Item '{potential_path}' still not found. Cannot zip.\n")
                        path_to_zip_item = None
                        content_name = None
                        download_successful = False
            elif len(new_items) == 0:
                yield from yield_update(f"⚠️ No new top-level files/folders detected in '{download_directory}'.\n   Possible reasons: Download failed, content merged into existing folder, or FS delay (esp. GDrive).\n   Cannot automatically zip.\n")
                download_successful = False
            else: # len(new_items) > 1
                yield from yield_update(f"⚠️ Multiple new items detected: {new_items}\n   Cannot automatically determine the single item to zip.\n   Please zip manually if needed.\n")
                download_successful = False

        # --- 4. Execute Zipping ---
        effective_auto_zip = auto_zip_after_download and zip_available

        if download_successful and effective_auto_zip and path_to_zip_item and content_name:
            yield from yield_update("\n--- Starting Zipping Process ---\n")

            parent_dir_to_cd_into = download_directory
            output_zip_dir_default = "/content/"
            os.makedirs(output_zip_dir_default, exist_ok=True)
            output_zip_path = None

            if output_zip_name_override:
                safe_override_name = os.path.basename(output_zip_name_override.strip('/\\'))
                is_drive_path = output_zip_name_override.startswith('/content/drive/')

                if is_drive_path:
                     if drive_mounted:
                         output_zip_path = output_zip_name_override
                         output_zip_dir = os.path.dirname(output_zip_path)
                         try:
                             os.makedirs(output_zip_dir, exist_ok=True)
                             yield from yield_update(f"✅ Ensured GDrive output directory exists: {output_zip_dir}\n")
                         except OSError as e:
                             err_msg = f"❌ ERROR: Could not create GDrive output directory for zip: {output_zip_dir}\n   Error details: {e}\n   Saving to /content/ instead.\n"
                             yield from yield_update(err_msg)
                             output_zip_path = os.path.join(output_zip_dir_default, safe_override_name if safe_override_name else f"{content_name}.zip") # Fallback
                         except Exception as e:
                             err_msg = f"❌ Unexpected error creating GDrive zip output directory: {e}\n   Saving to /content/ instead.\n"
                             yield from yield_update(err_msg)
                             output_zip_path = os.path.join(output_zip_dir_default, safe_override_name if safe_override_name else f"{content_name}.zip") # Fallback
                     else:
                          yield from yield_update(f"⚠️ WARNING: Specified output zip path '{output_zip_name_override}' is on Google Drive, but Drive is not mounted. Saving to /content/ instead.\n")
                          output_zip_path = os.path.join(output_zip_dir_default, safe_override_name if safe_override_name else f"{content_name}.zip") # Fallback

                elif safe_override_name: # Just a filename (or path not in drive treated as filename)
                    output_zip_path = os.path.join(output_zip_dir_default, safe_override_name)
                    if '/' in output_zip_name_override.strip('/\\') and not is_drive_path:
                         yield from yield_update(f"⚠️ Interpreting path '{output_zip_name_override}' as filename in {output_zip_dir_default}.\n")
                else: # Override was empty or invalid
                     yield from yield_update(f"⚠️ Invalid zip name override. Using default name.\n")
                     output_zip_path = os.path.join(output_zip_dir_default, f"{content_name}.zip")

            else: # Default name
                output_zip_path = os.path.join(output_zip_dir_default, f"{content_name}.zip")


            if output_zip_path:
                yield from yield_update(f"ℹ️ Source Item for Zipping: '{path_to_zip_item}'\n")
                yield from yield_update(f"ℹ️ Target Zip File: '{output_zip_path}'\n")

                if os.path.exists(output_zip_path):
                    yield from yield_update(f"⚠️ Warning: Output file '{output_zip_path}' already exists. Overwriting.\n")

                # Execute Zipping
                quoted_output_path = shlex.quote(output_zip_path)
                quoted_item_name = shlex.quote(content_name)
                quoted_parent_dir = shlex.quote(parent_dir_to_cd_into)

                yield from yield_update(f"\n⏳ Zipping '{content_name}' from '{parent_dir_to_cd_into}'...\n")

                zip_command = f'cd {quoted_parent_dir} && zip -rqo {quoted_output_path} {quoted_item_name}'
                start_time = time.time()
                zip_process = subprocess.run(zip_command, shell=True, capture_output=True, text=True)
                end_time = time.time()
                zip_duration = end_time - start_time

                yield from yield_update("-" * 30 + "\n")

                # Add extra check/wait if writing zip to GDrive
                zip_exists = False
                if output_zip_path.startswith('/content/drive'):
                    yield from yield_update(f"⏳ Verifying zip file creation on Google Drive (may take a moment)...\n")
                    time.sleep(5) # Initial wait
                    if os.path.exists(output_zip_path):
                         zip_exists = True
                    else: # Wait longer if not found immediately
                         time.sleep(10)
                         zip_exists = os.path.exists(output_zip_path)
                else:
                    zip_exists = os.path.exists(output_zip_path)


                if zip_process.returncode == 0 and zip_exists:
                    final_zip_path = output_zip_path
                    yield from yield_update(f"✅ Successfully created zip file: '{output_zip_path}'\n")
                    yield from yield_update(f"   Time taken: {zip_duration:.2f} seconds\n")
                    try:
                        # Add retry for GDrive size check
                        file_size = -1
                        for attempt in range(3):
                             try:
                                 file_size = os.path.getsize(output_zip_path)
                                 if file_size >= 0: break # Got a valid size
                             except OSError:
                                 if attempt < 2 : time.sleep(3) # Wait before retrying on GDrive
                                 else: raise # Raise error on last attempt
                        if file_size < 0 : raise OSError("Could not get file size")

                        if file_size < 1024: size_str = f"{file_size} B"
                        elif file_size < 1024**2: size_str = f"{file_size / 1024:.2f} KB"
                        elif file_size < 1024**3: size_str = f"{file_size / 1024**2:.2f} MB"
                        else: size_str = f"{file_size / 1024**3:.2f} GB"
                        yield from yield_update(f"   Size: {size_str}\n")
                    except OSError as e:
                        yield from yield_update(f"   Could not retrieve file size after retries: {e}\n")

                    yield from yield_update(
                        "✅ Zip file is ready for download below.\n",
                        new_zip_update=gr.update(value=final_zip_path, visible=True)
                    )

                else:
                    yield from yield_update(f"❌ Error: Zip file creation failed or file not found after process completion.\n")
                    yield from yield_update(f"   Zip Process Exit Code: {zip_process.returncode}\n")
                    yield from yield_update(f"   File Found Check ('{output_zip_path}'): {zip_exists}\n")
                    if zip_process.stderr: yield from yield_update(f"   Stderr:\n{zip_process.stderr.strip()}\n")
                    if zip_process.stdout: yield from yield_update(f"   Stdout:\n{zip_process.stdout.strip()}\n")


        # --- Handle Skipped Zipping Cases ---
        # (Logic remains the same)
        elif auto_zip_after_download and not effective_auto_zip:
             yield from yield_update("\n--- Zipping Skipped ---\n")
             yield from yield_update(f"ℹ️ Zipping skipped: 'zip' utility not available or installation failed.\n")
        elif auto_zip_after_download and not path_to_zip_item:
            yield from yield_update("\n--- Zipping Skipped ---\n")
            yield from yield_update("ℹ️ Zipping skipped: Could not reliably determine the single item to zip.\n")
        elif not download_successful:
            yield from yield_update("\n--- Zipping Skipped ---\n")
            yield from yield_update("ℹ️ Zipping skipped: Download failed or did not complete successfully.\n")
        elif not auto_zip_after_download:
             yield from yield_update("\n--- Zipping Skipped ---\n")
             yield from yield_update("ℹ️ Automatic zipping was disabled by user setting.\n")

        yield from yield_update("\n--- Automation Finished ---\n")

        return status_log, gr.update(value=final_zip_path, visible=final_zip_path is not None)

    except Exception as e:
        error_details = traceback.format_exc()
        status_log += f"\n\n❌❌❌ UNEXPECTED CRITICAL ERROR ❌❌❌\n"
        status_log += f"An error occurred outside the main download/zip flow:\n{e}\n"
        status_log += f"Traceback:\n{error_details}\n"
        status_log += "\n--- Automation Stopped Due to Error ---\n"
        # Ensure yield returns correct tuple structure even on unexpected exit
        yield status_log, gr.update(value=None, visible=False)
        # Final return for safety
        return status_log, gr.update(value=None, visible=False)


# ==============================================================================
# Gradio Interface Definition (Updated Markdown)
# ==============================================================================

# Create dynamic markdown text based on mount status
mount_status_md = """
**Google Drive Status:** {}
{}
---
Upload a `.torrent` file, specify download and zip options, and start the process.
Status updates will appear below. If zipping is successful, a download button for the zip file will appear.
"""
if drive_mounted:
    mount_status_md = mount_status_md.format(
        "✅ Mounted.",
        "You can use paths like `/content/drive/MyDrive/...`."
        )
else:
     mount_status_md = mount_status_md.format(
        "⚠️ Not Mounted.",
        "Using paths starting with `/content/drive/MyDrive/...` will likely **FAIL**. Mount Drive in a separate cell first if needed, then **rerun this app cell**."
        )


with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(f"""# Torrent Downloader & Zipper""") # Title
    gr.Markdown(mount_status_md) # Dynamic status message

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 1. Torrent Source")
            torrent_file_input = gr.File(label="Upload .torrent File", file_types=[".torrent"])

            gr.Markdown("### 2. Download Location & Options")
            download_dir_input = gr.Textbox(
                label="Base Download Directory (Optional)",
                placeholder="/content/drive/MyDrive/TorrentDownloads or blank for /content/downloads",
                info="Leave blank for temporary '/content/downloads'. Use Drive paths only if mounted."
            )
            stop_seeding_input = gr.Checkbox(label="Stop Seeding Immediately After Download", value=True)

        with gr.Column(scale=1):
            gr.Markdown("### 3. Zipping Options")
            auto_zip_input = gr.Checkbox(label="Enable Automatic Zipping After Download", value=True)
            zip_name_input = gr.Textbox(
                label="Output Zip File Name/Path (Optional)",
                placeholder="my_archive.zip or /content/drive/MyDrive/Zips/archive.zip",
                info="If blank, saves <item_name>.zip to /content/. Use Drive paths only if mounted."
            )

    with gr.Row():
        start_button = gr.Button("Start Download & Zip", variant="primary")

    with gr.Row():
        status_output = gr.Textbox(label="Status Log", lines=15, interactive=False, autoscroll=True)

    with gr.Row():
         zip_download_output = gr.File(label="Download Generated Zip File", visible=False, interactive=False)


    # --- Button Click Action ---
    start_button.click(
        fn=run_download_and_zip,
        inputs=[
            torrent_file_input,
            download_dir_input,
            stop_seeding_input,
            auto_zip_input,
            zip_name_input
        ],
        outputs=[
            status_output,
            zip_download_output
        ]
    )


# --- Launch the Gradio App ---
# demo.close() # Use if needed to close previous instances when rerunning
demo.launch(debug=True)

--- Google Drive Mounting ---
Mounted at /content/drive
✅ Google Drive successfully mounted at: /content/drive
✅ Found expected path: /content/drive/MyDrive
------------------------------
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://49841f1e737caea17a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
